In [ ]:
!pip install pydriller

     |████████████████████████████████| 65 kB 2.5 MB/s 
     |████████████████████████████████| 64 kB 3.2 MB/s 
     |████████████████████████████████| 180 kB 41.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 51.6 MB/s 
     |████████████████████████████████| 895 kB 41.9 MB/s 
     |████████████████████████████████| 3.3 MB 51.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


**Import necessary libraries**

In [ ]:
#for data
import pandas as pd 

#for github scraping
import os
import requests
import time
import sys
import json
from requests_oauthlib import OAuth1Session
from requests_oauthlib import OAuth1

#text preprocessing
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[All the Things You Can Do With GitHub API and Python](https://towardsdatascience.com/all-the-things-you-can-do-with-github-api-and-python-f01790fca131)

In [ ]:


file_no=1

def gettoken():
  with open('/drive/MyDrive/token.txt', 'r') as token:
        gittoken = token.readline().replace("\n","")
  return gittoken

#print(gettoken())

def searchgit(fixes,commits):
    
    params = (
      ('q', fixes),('per_page',100) )
   
    headers = {'Accept': 'application/vnd.github.cloak-preview', 'Authorization': 'token ' + gettoken()}
    url = "https://api.github.com/search/commits"
  
    for i in range(0,100):
        print(i,fixes, commits)
        limit = 0
        while(limit == 0):
            response = requests.get(url,headers=headers,params=(('q',fixes),('per_page',100)))
            head = response.headers      
            if 'X-RateLimit-Remaining' in head:
                limit = int(head['X-RateLimit-Remaining'])
                if limit == 0:
                # Limit of requests per time was reached, sleep to wait until we can request again
                    print("Rate limit. Sleep.")
                    global file_no
                    file_no+=1
                    #file_name = '/drive/MyDrive/git_commits_XSS'+str(file_no)+'.json'
                    #file_name = '/drive/MyDrive/git_commits_SQL'+str(file_no)+'.json'
                    #file_name = '/drive/MyDrive/git_commits_Buffer'+str(file_no)+'.json'
                    file_name = '/drive/MyDrive/git_commits_Improperaccess'+str(file_no)+'.json'
                    print(file_name)
                    content= response.json()
                    with open(file_name, 'w') as commitfile:
                        json.dump(content, commitfile)  
                    time.sleep(35)  
        if 'Link' not in head:
            break;                  
                    
   
    
    # for j in range(0, len(content)):
    #     repo = content["items"][j]["repository"]["html_url"]
    

   


  
  


In [ ]:
commits = {}

#load previously scraped commits
'''
with open('/drive/MyDrive/git_commits_XSS.json','r') as jsonfile:
    data = jsonfile.read()
    commits= json.loads(data.decode("utf-8"))
'''   
#print(data)
    
#vulnerabilities = ["XSS","cross site","cross site request forgery"]   
#vulnerabilities = ["SQL","SQLI","injection"]
#vulnerabilities =["buffer","overflow","stack"]
vulnerabilities =["improper","unauthenticated","gain access","permission"]
#fixwords = ["prevent", "fix", "attack", "protect", "issue", "correct", "update", "improve", "change", "check", "malicious", "insecure", "vulnerable", "vulnerability"]
fixwords = ["malicious", "insecure", "vulnerable", "vulnerability","vulnerabilities"]

In [ ]:


#for all combinations of vulnerabilities and fixwords, scrape github 
for v in vulnerabilities:
  for fix in fixwords:
      searchgit(v + " " + fix, commits ); 

In [ ]:
import glob

countxss = 0
countsqli = 0
countimproperaccess = 0
for file in glob.glob('/drive/MyDrive/git_commits_XSS*.json'):
    countxss+=1
    print(file)

for file in glob.glob('/drive/MyDrive/git_commits_SQL*.json'):
    countsqli+=1

for file in glob.glob('/drive/MyDrive/git_commits_Improperaccess*.json'):
    countimproperaccess+=1
#
# print(countxss)
# print(countsqli)
# print(countimproperaccess)

In [ ]:
countxss

58

In [ ]:
#extracting messages for xss

xss_msg=list()
filename_xss = list()

for i in range(2,countxss+2):
  with open('/drive/MyDrive/git_commits_XSS'+str(i)+'.json')as inputfile:
    data = json.load(inputfile)
    if i==5 or i==46 or i==49: #file numbers without any data because of limit exceeded error
      continue
    else:
     # print(i, len(data["items"]))
     for j in range(0,len(data["items"])):
       filename_xss.append( '/drive/MyDrive/git_commits_XSS'+str(i)+'.json')
       xss_msg.append(data["items"][j]["commit"]["message"])
#print(type(data))

final_data_xss= {'message':xss_msg,'type':'XSS','filename':filename_xss}
data_frame_xss = pd.DataFrame(final_data_xss)

#with open(/drive/MyDrive/git_commits_XSS'+str(i)+'.json)

In [ ]:
data_frame_xss

,message,type,filename
0,Add parameterized queries to protect against m...,XSS,/drive/MyDrive/git_commits_XSS2.json
1,Replace malicious sql query\n\nLet's show an a...,XSS,/drive/MyDrive/git_commits_XSS2.json
2,Escaped query values to prevent SQL injection ...,XSS,/drive/MyDrive/git_commits_XSS2.json
3,setup a defence for malicious SQL injection,XSS,/drive/MyDrive/git_commits_XSS2.json
4,ecure/prevent sql malicious injection,XSS,/drive/MyDrive/git_commits_XSS2.json
...,...,...,...
5495,[NSE] Added http-csrf.nse that detects Cross S...,XSS,/drive/MyDrive/git_commits_XSS59.json
5496,[NSE] Added http-csrf.nse that detects Cross S...,XSS,/drive/MyDrive/git_commits_XSS59.json
5497,[NSE] Added http-csrf.nse that detects Cross S...,XSS,/drive/MyDrive/git_commits_XSS59.json
5498,[NSE] Added http-csrf.nse that detects Cross S...,XSS,/drive/MyDrive/git_commits_XSS59.json


In [ ]:
countsqli

44

In [ ]:
#extracting messages for sql

sql_msg=list()
filename_sql = list()

for i in range(2,countsqli+2):
  with open('/drive/MyDrive/git_commits_SQL'+str(i)+'.json')as inputfile:
    data = json.load(inputfile)
    for j in range(0,len(data["items"])):
      filename_sql.append( '/drive/MyDrive/git_commits_SQL'+str(i)+'.json')
      sql_msg.append(data["items"][j]["commit"]["message"])

final_data_sql= {'message':sql_msg,'type':'SQLI','filename':filename_sql}
data_frame_sql = pd.DataFrame(final_data_sql)


In [ ]:
#extracting messages for improper access

ia_msg=list()
filename_ia = list()

for i in range(2,countimproperaccess+2):
  with open('/drive/MyDrive/git_commits_Improperaccess'+str(i)+'.json')as inputfile:
    data = json.load(inputfile)
    if i==40:  #file numbers to ignore
      continue
    else:
      #print(i, len(data["items"]))
      for j in range(0,len(data["items"])):
        filename_ia.append( '/drive/MyDrive/git_commits_Improperaccess'+str(i)+'.json')
        ia_msg.append(data["items"][j]["commit"]["message"])

final_data_ia= {'message':ia_msg,'type':'ImproperAccess','filename':filename_ia}
data_frame_ia = pd.DataFrame(final_data_ia)


In [ ]:
data_frame_ia

,message,type,filename
0,Iranian Government-Sponsored APT Cyber Actors ...,ImproperAccess,/drive/MyDrive/git_commits_Improperaccess2.json
1,exiv2: Fix CVE-2021-3482\n\n References\n...,ImproperAccess,/drive/MyDrive/git_commits_Improperaccess2.json
2,exiv2: Fix CVE-2021-3482\n\n References\n...,ImproperAccess,/drive/MyDrive/git_commits_Improperaccess2.json
3,exiv2: Fix CVE-2021-3482\n\n References\n...,ImproperAccess,/drive/MyDrive/git_commits_Improperaccess2.json
4,exiv2: Fix CVE-2021-3482\n\n References\n...,ImproperAccess,/drive/MyDrive/git_commits_Improperaccess2.json
...,...,...,...
6195,Fix security vulnerability in DPMS\n\nChanged ...,ImproperAccess,/drive/MyDrive/git_commits_Improperaccess64.json
6196,Fix security vulnerability in DPMS\n\nChanged ...,ImproperAccess,/drive/MyDrive/git_commits_Improperaccess64.json
6197,Fix security vulnerability in DPMS\n\nChanged ...,ImproperAccess,/drive/MyDrive/git_commits_Improperaccess64.json
6198,Fix security vulnerability in DPMS\n\nChanged ...,ImproperAccess,/drive/MyDrive/git_commits_Improperaccess64.json


In [ ]:
#Remove duplicate values if any from data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


AttributeError: ignored

In [ ]:
data_frame_xss

,message,type,filename
0,Add parameterized queries protect malicious SQ...,XSS,/drive/MyDrive/git_commits_XSS2.json
1,Replace malicious sql query Let's show actual ...,XSS,/drive/MyDrive/git_commits_XSS2.json
2,Escaped query values prevent SQL injection ins...,XSS,/drive/MyDrive/git_commits_XSS2.json
3,setup defence malicious SQL injection,XSS,/drive/MyDrive/git_commits_XSS2.json
4,ecure/prevent sql malicious injection,XSS,/drive/MyDrive/git_commits_XSS2.json
...,...,...,...
5495,[NSE] Added http-csrf.nse detects Cross Site R...,XSS,/drive/MyDrive/git_commits_XSS59.json
5496,[NSE] Added http-csrf.nse detects Cross Site R...,XSS,/drive/MyDrive/git_commits_XSS59.json
5497,[NSE] Added http-csrf.nse detects Cross Site R...,XSS,/drive/MyDrive/git_commits_XSS59.json
5498,[NSE] Added http-csrf.nse detects Cross Site R...,XSS,/drive/MyDrive/git_commits_XSS59.json


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from datetime import datetime
from pydriller import Repository
import json

import os.path

with open('/drive/MyDrive/git_commits.json','r') as jsonfile:
    data = jsonfile.read()
  
print(data[0:100])

# tokenizer = AutoTokenizer.from_pretrained('mrm8488/codebert-base-finetuned-detect-insecure-code')
# model = AutoModelForSequenceClassification.from_pretrained('mrm8488/codebert-base-finetuned-detect-insecure-code')

# currenttime = datetime.now()
# formatcurrtime = currenttime.strftime("%H:%M")
# print("Loading done. ",formatcurrtime)

# for i in data:
#   project_url = data[i]["html_url"]
#   for commit in Repository(project_url).traverse_commits:
#     for m in commit.modified_files:
#       inputs = tokenizer(str(m),return_tensors ="pt", truncation=True, padding='max_length')
#       labels = torch.tensor([1]).unsqueeze(0)
#       outputs = model(**inputs, labels = label)
#       loss = outputs.loss
#       logits = outputs.logits
#       if np.argmax(logits.detach().numpy)==0:
#         print(i["html_url"]," is vulnerable")

{"documentation_url": "https://docs.github.com/en/free-pro-team@latest/rest/overview/resources-in-th


**References**:
1. https://towardsdatascience.com/introduction-to-git-data-extraction-and-analysis-in-python-e7e2bf9b4606
2. https://towardsdatascience.com/text-pre-processing-stop-words-removal-using-different-libraries-f20bac19929a
3. https://towardsdatascience.com/nlp-preprocessing-with-nltk-3c04ee00edc0
4. https://medium.com/swlh/how-to-train-word2vec-model-using-gensim-library-115b35440c90

5. Keywords related to vulnerabilities: [Identifying the characteristics of vulnerable code changes: an empirical study](https://www.researchgate.net/figure/Keywords-Associated-with-Vulnerabilities_tbl1_286077750)